# 给翅膀上的黑点加上box

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import json, sys
from pathlib import Path
from typing import List, Dict, Any, Tuple

BOX_SIZE = 60          # 你要的 10 像素正方形
REMOVE_EXISTING = True # 生成前移除已有的 LB/RB，避免重复

def clamp(v, lo, hi): 
    return max(lo, min(v, hi))

def make_rect_points(cx: float, cy: float, w: float, h: float, img_w: int, img_h: int):
    x1 = clamp(cx - w/2, 0, img_w - 1)
    y1 = clamp(cy - h/2, 0, img_h - 1)
    x2 = clamp(cx + w/2, 0, img_w - 1)
    y2 = clamp(cy + h/2, 0, img_h - 1)
    # 按 LabelMe 常见顺序：左上、右上、右下、左下
    return [[float(x1), float(y1)], [float(x2), float(y1)],
            [float(x2), float(y2)], [float(x1), float(y2)]]

def rect_bounds(pts: List[List[float]]) -> Tuple[float,float,float,float]:
    xs = [p[0] for p in pts]
    ys = [p[1] for p in pts]
    return min(xs), min(ys), max(xs), max(ys)

def point_in_rect(x: float, y: float, rect_pts: List[List[float]]) -> bool:
    x1, y1, x2, y2 = rect_bounds(rect_pts)
    return (x1 - 1e-6) <= x <= (x2 + 1e-6) and (y1 - 1e-6) <= y <= (y2 + 1e-6)

def normalize_label(s: str) -> str:
    return str(s or "").strip().lower()

def build_box_shape(label: str, pts: List[List[float]], src: str, idx: int, attach_swd: int = None) -> Dict[str, Any]:
    attr = {"from_point": src, "point_index": idx}
    if attach_swd is not None:
        attr["attach_swd_index"] = attach_swd
    return {
        "label": label,           # "LB" 或 "RB"
        "score": 1.0,
        "points": pts,
        "group_id": 0,
        "description": f"auto-from-{src}#{idx}",
        "difficult": False,
        "shape_type": "rectangle",
        "flags": {},
        "attributes": attr,
        "kie_linking": []
    }

def add_boxes_to_json(p: Path, out: Path = None):
    data = json.loads(p.read_text(encoding="utf-8"))

    img_w = data.get("imageWidth")
    img_h = data.get("imageHeight")
    assert isinstance(img_w, int) and isinstance(img_h, int), f"缺少 imageWidth/Height: {p}"

    shapes: List[Dict[str, Any]] = list(data.get("shapes", []))

    # 可选：先移除已有 LB/RB，避免重复
    if REMOVE_EXISTING:
        shapes = [s for s in shapes if normalize_label(s.get("label")) not in ("lb", "rb")]

    # 收集所有 swd 矩形（用于可选的归属标记）
    swd_rects = []
    for s in shapes:
        if normalize_label(s.get("label")) == "swd" and s.get("shape_type") == "rectangle":
            pts = s.get("points") or []
            if isinstance(pts, list) and len(pts) >= 4:
                swd_rects.append(pts)

    # 收集所有 lp / rp 点
    lps, rps = [], []
    for s in shapes:
        if s.get("shape_type") == "point" and s.get("points"):
            lab = normalize_label(s.get("label"))
            (x, y) = s["points"][0]
            if lab == "lp":
                lps.append((float(x), float(y)))
            elif lab == "rp":
                rps.append((float(x), float(y)))

    # 为每个 lp 生成 LB
    for i, (x, y) in enumerate(lps, start=1):
        # 找到是否落在某个 swd 矩形内（记录索引，便于回溯）
        attach = None
        for k, rect in enumerate(swd_rects, start=1):
            if point_in_rect(x, y, rect):
                attach = k
                break
        pts = make_rect_points(x, y, BOX_SIZE, BOX_SIZE, img_w, img_h)
        shapes.append(build_box_shape("p", pts, "lp", i, attach))

    # 为每个 rp 生成 RB
    for i, (x, y) in enumerate(rps, start=1):
        attach = None
        for k, rect in enumerate(swd_rects, start=1):
            if point_in_rect(x, y, rect):
                attach = k
                break
        pts = make_rect_points(x, y, BOX_SIZE, BOX_SIZE, img_w, img_h)
        shapes.append(build_box_shape("p", pts, "rp", i, attach))

    data["shapes"] = shapes
    out = out or p
    out.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")

if __name__ == "__main__":
    # 用法：
    # 1) 单文件：python add_boxes_multi.py path/to/one.json
    # 2) 目录  ：python add_boxes_multi.py path/to/dir
    target = Path("/workspace/models/runs_yolov11_pose/cropped_objects_v5/swd")
    files = [target] if target.suffix.lower()==".json" else sorted(target.glob("*.json"))
    for f in files:
        add_boxes_to_json(f)
    print(f"Done. processed {len(files)} file(s).")


Done. processed 849 file(s).
